In [1]:
import torch
import logging
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
from tree_parser import repo_scan_parser, build_mamba_prompt

/home/quinyang/whole_codebase_auditor/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration
MODEL_ID = "Zyphra/Zamba2-2.7B"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
print(f"⚙️  Loading Zamba2 (Hybrid Mamba+Attention) on {DEVICE}...")

# Set logging to see download progress
logging.basicConfig(level=logging.INFO)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map=DEVICE,
        dtype=torch.bfloat16,       # Changed from 'torch_dtype' to 'dtype'
        trust_remote_code=True
    )
    print("✅ Zamba2 Loaded Successfully!")
except ImportError:
    print("❌ Error: Missing 'mamba-ssm' or 'causal-conv1d'.")
except ValueError as e:
    print(f"❌ Configuration Error: {e}")

⚙️  Loading Zamba2 (Hybrid Mamba+Attention) on cuda...


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.21s/it]


✅ Zamba2 Loaded Successfully!


In [4]:
print("\n🚀 Scanning Repository...")
scanned_results = repo_scan_parser()
full_prompt = build_mamba_prompt(scanned_results)


🚀 Scanning Repository...
🔹 Authenticate with GitHub...
🔹 Fetching file list from branch: V2.1...
Found 15 files in the repository.

⏭️ Skipping unsupported file: .env
⏭️ Skipping unsupported file: .gitignore
✅ Loaded grammar: CPP
🔹 Processing file (3/15): HPC/heston_gpu.cu
✅ Loaded grammar: PYTHON
🔹 Processing file (4/15): HPC/heston_server.py
🔹 Processing file (5/15): HPC/stock_calibration.py
🔹 Processing file (6/15): ML/generate_training_data.py
🔹 Processing file (7/15): ML/improved_parameter_variation.py
🔹 Processing file (8/15): ML/ml_data_loader.py
🔹 Processing file (9/15): ML/ml_model.py
⏭️ Skipping unsupported file: ML/mlp.ipynb
⏭️ Skipping unsupported file: README.md
⏭️ Skipping known massive vendor file: include/json.hpp
⏭️ Skipping unsupported file: requirements.txt
⏭️ Skipping unsupported file: training_data/metadata.json
⏭️ Skipping unsupported file: training_data/normalization_stats.json

✅ Scan Complete. Processed 7 files.
📊 Total Codebase Size: 90.91 KB


In [5]:
# 4. Tokenize & Generate
print(f"\n🧠 Processing {len(full_prompt)} characters...")
input_ids = tokenizer(full_prompt, return_tensors="pt").input_ids.to(DEVICE)
    
print("🕵️‍♂️ WCA Auditor is thinking...")
start_time = time.time()


🧠 Processing 93834 characters...
🕵️‍♂️ WCA Auditor is thinking...


In [6]:
with torch.inference_mode():
    outputs = model.generate(
        input_ids, 
        max_new_tokens=1024, 
        do_sample=True,
        temperature=0.4,
        use_cache=True 
    )

This is a friendly reminder - the current text generation call has exceeded the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


RuntimeError: CUDA error: unknown error
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print("\n" + "="*50)
print("🛡️  AUDIT REPORT")
print("="*50)
# Decode only the NEW tokens
print(tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True))
print(f"\n⏱️  Time: {time.time() - start_time:.2f}s")